## <font color=009DD2> <span style="background-color: #A9C4C0"> Imports

In [1]:
import sys,json
import glob, os
sys.path.append('/media/miplab-nas2/Data3/BMPD/hc_project/analysis/code/')

from seed_to_voxels_inprogress_nk import Seed2voxels

%matplotlib inline
%load_ext autoreload
%autoreload 2

## <font color=009DD2> <span style="background-color: #A9C4C0"> Initialization

### <font color=009DD2> I Configuration & parameters </font>

In [2]:
with open('../config/config_NK.json') as config_file:
    config = json.load(config_file) # load config file

# Parameters (maybe we should move them to config?)
signal='ai'
seed_names=['icap1','icap2','icap3','icap4','icap5','icap6','icap7','icap8','icap9'] # Define seed names (here, icap numbers)
target_name=['brain'] # Define structure to use as target (brain or spinal cord)
ktot = 9 # Total number of components to use (just to define path)


### <font color=009DD2> II Select files </font>

In [3]:
# One target per subject
data_target=[];ts_target_txt=[]
for subject_name in config['list_subjects']:
    data_target.append(config['main_dir'] + '1_nilearn-based/3_func_smoothed/sub-'+ subject_name + '/TA_results/MNI/TotalActivation/Activity_inducing.nii')
    ts_target_txt.append(config['main_dir'] + config['seed2vox_dir'] + '/' +target_name[0]+'_all/timeseries/sub-' + subject_name + '_' + target_name[0] + '_all_timeseries_ai')


# But possible to have multiple seeds (so create a dictionary to store list of each seed)
ts_seed_txt={}    
for seed_name in seed_names:
    ts_seed_txt[seed_name] = []
    for subject_name in config['list_subjects']:
        ts_seed_txt[seed_name].append(config['main_dir'] + config['seed2vox_dir'] + '/spinalcord_icaps_k' + str(ktot) +'/'+seed_name+'/timeseries/sub-' + subject_name + '_' + seed_name + '_timeseries_ai')
    if not os.path.exists(config['main_dir'] + config['seed2vox_dir'] + '/spinalcord_icaps_k' + str(ktot) + '/' +seed_name+'/'+target_name[0]+'_fc_maps/'):
        os.mkdir(config['main_dir'] + config['seed2vox_dir'] + '/spinalcord_icaps_k' + str(ktot) +'/'+seed_name+'/'+target_name[0]+'_fc_maps/')

mask_target=config['main_dir'] + config['masks'][target_name[0].split('_')[0]]
 
# Create directories if needed
if not os.path.exists(config['main_dir'] + config['seed2vox_dir'] + '/' +target_name[0]+'_all/timeseries/'):
    os.mkdir(config['main_dir'] + config['seed2vox_dir'] + '/' +target_name[0]+'_all')
    os.mkdir(config['main_dir'] + config['seed2vox_dir'] + '/' +target_name[0]+'_all/timeseries/') # folder to store timeseries extraction

### <font color=009DD2> III Initialize class based on this </font>

In [4]:
seed2voxels=Seed2voxels(config,signal,seed_names,target_name) # initialize the function

## <font color=009DD2> <span style="background-color: #A9C4C0"> Data extraction


### <font color=009DD2> I. Time series extraction - Seed </font>

In [5]:
seed_timeseries={};
for seed_name in seed_names:
    seed_timeseries[seed_name]=[]
    seed_timeseries[seed_name]=seed2voxels.extract_data(timeseries_txt=ts_seed_txt[seed_name],
                                         run="load",n_jobs=8) # Load iCAP timecourses as seed

### <font color=009DD2> II. Time series extraction - Target </font>

In [6]:
target_timeseries=seed2voxels.extract_data(img=data_target, mask=mask_target, timeseries_txt=ts_target_txt,
                run="extract",n_jobs=8) # Load voxelwise brain AI as target

Note: when using "ai", signals can only be extracted for targets.


## <font color=009DD2> <span style="background-color: #A9C4C0"> Correlation analysis

In [7]:
output_img={};
for seed_name in seed_names:
    output_img[seed_name]=[]
    output_img[seed_name]=config['main_dir'] + config['seed2vox_dir'] + '/spinalcord_icaps_k' + str(ktot) + '/'+seed_name+'/'+target_name[0]+'_fc_maps/' + str(len(config['list_subjects'])) + 'subjects_' + seed_name
    seed2voxels.correlation_maps(seed_timeseries[seed_name],target_timeseries,mask=mask_target,output_img=output_img[seed_name],Fisher=True,n_jobs=8)

/home/kinany/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/kinany/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2643: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/home/kinany/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/kinany/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2643: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/home/kinany/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/kinany/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2643: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/home/kinany/ana

## <font color=009DD2> <span style="background-color: #A9C4C0"> Statistics

### <font color=009DD2> Using non-parametric permutation testing on smoothed images</font>

In [201]:
run_string = 'randomise -i ' + output_img['icap2'] + '_zcorr.nii -o ' + output_img['icap1'] + ' -1 -T -m ' + mask_target
os.system(run_string)

randomise options: -i /media/miplab-nas2/Data3/BMPD/hc_project//seed_to_voxels//spinalcord_icaps_k9/icap2/brain_fc_maps/22subjects_icap2_zcorr.nii -o /media/miplab-nas2/Data3/BMPD/hc_project//seed_to_voxels//spinalcord_icaps_k9/icap1/brain_fc_maps/22subjects_icap1 -1 -T -m /media/miplab-nas2/Data3/BMPD/hc_project//templates/MNI_hc_project/MNI_T1w_GM_AverageHCs_3mm_bin_corrected.nii 
Loading Data: 
Data loaded
One-sample design detected; sign-flipping instead of permuting.
4.1943e+06 sign-flips required for exhaustive test of t-test 1
Doing 5000 random permutations
Starting permutation 1 (Unpermuted data)
Starting permutation 2
Starting permutation 3
Starting permutation 4
Starting permutation 5
Starting permutation 6
Starting permutation 7
Starting permutation 8
Starting permutation 9
Starting permutation 10
Starting permutation 11
Starting permutation 12
Starting permutation 13
Starting permutation 14
Starting permutation 15
Starting permutation 16
Starting permutation 17
Starting per

Starting permutation 310
Starting permutation 311
Starting permutation 312
Starting permutation 313
Starting permutation 314
Starting permutation 315
Starting permutation 316
Starting permutation 317
Starting permutation 318
Starting permutation 319
Starting permutation 320
Starting permutation 321
Starting permutation 322
Starting permutation 323
Starting permutation 324
Starting permutation 325
Starting permutation 326
Starting permutation 327
Starting permutation 328
Starting permutation 329
Starting permutation 330
Starting permutation 331
Starting permutation 332
Starting permutation 333
Starting permutation 334
Starting permutation 335
Starting permutation 336
Starting permutation 337
Starting permutation 338
Starting permutation 339
Starting permutation 340
Starting permutation 341
Starting permutation 342
Starting permutation 343
Starting permutation 344
Starting permutation 345
Starting permutation 346
Starting permutation 347
Starting permutation 348
Starting permutation 349


Starting permutation 638
Starting permutation 639
Starting permutation 640
Starting permutation 641
Starting permutation 642
Starting permutation 643
Starting permutation 644
Starting permutation 645
Starting permutation 646
Starting permutation 647
Starting permutation 648
Starting permutation 649
Starting permutation 650
Starting permutation 651
Starting permutation 652
Starting permutation 653
Starting permutation 654
Starting permutation 655
Starting permutation 656
Starting permutation 657
Starting permutation 658
Starting permutation 659
Starting permutation 660
Starting permutation 661
Starting permutation 662
Starting permutation 663
Starting permutation 664
Starting permutation 665
Starting permutation 666
Starting permutation 667
Starting permutation 668
Starting permutation 669
Starting permutation 670
Starting permutation 671
Starting permutation 672
Starting permutation 673
Starting permutation 674
Starting permutation 675
Starting permutation 676
Starting permutation 677


Starting permutation 968
Starting permutation 969
Starting permutation 970
Starting permutation 971
Starting permutation 972
Starting permutation 973
Starting permutation 974
Starting permutation 975
Starting permutation 976
Starting permutation 977
Starting permutation 978
Starting permutation 979
Starting permutation 980
Starting permutation 981
Starting permutation 982
Starting permutation 983
Starting permutation 984
Starting permutation 985
Starting permutation 986
Starting permutation 987
Starting permutation 988
Starting permutation 989
Starting permutation 990
Starting permutation 991
Starting permutation 992
Starting permutation 993
Starting permutation 994
Starting permutation 995
Starting permutation 996
Starting permutation 997
Starting permutation 998
Starting permutation 999
Starting permutation 1000
Starting permutation 1001
Starting permutation 1002
Starting permutation 1003
Starting permutation 1004
Starting permutation 1005
Starting permutation 1006
Starting permutati

Starting permutation 1285
Starting permutation 1286
Starting permutation 1287
Starting permutation 1288
Starting permutation 1289
Starting permutation 1290
Starting permutation 1291
Starting permutation 1292
Starting permutation 1293
Starting permutation 1294
Starting permutation 1295
Starting permutation 1296
Starting permutation 1297
Starting permutation 1298
Starting permutation 1299
Starting permutation 1300
Starting permutation 1301
Starting permutation 1302
Starting permutation 1303
Starting permutation 1304
Starting permutation 1305
Starting permutation 1306
Starting permutation 1307
Starting permutation 1308
Starting permutation 1309
Starting permutation 1310
Starting permutation 1311
Starting permutation 1312
Starting permutation 1313
Starting permutation 1314
Starting permutation 1315
Starting permutation 1316
Starting permutation 1317
Starting permutation 1318
Starting permutation 1319
Starting permutation 1320
Starting permutation 1321
Starting permutation 1322
Starting per

...

Starting permutation 3840
Starting permutation 3841
Starting permutation 3842
Starting permutation 3843
Starting permutation 3844
Starting permutation 3845
Starting permutation 3846
Starting permutation 3847
Starting permutation 3848
Starting permutation 3849
Starting permutation 3850
Starting permutation 3851
Starting permutation 3852
Starting permutation 3853
Starting permutation 3854
Starting permutation 3855
Starting permutation 3856
Starting permutation 3857
Starting permutation 3858
Starting permutation 3859
Starting permutation 3860
Starting permutation 3861
Starting permutation 3862
Starting permutation 3863
Starting permutation 3864
Starting permutation 3865
Starting permutation 3866
Starting permutation 3867
Starting permutation 3868
Starting permutation 3869
Starting permutation 3870
Starting permutation 3871
Starting permutation 3872
Starting permutation 3873
Starting permutation 3874
Starting permutation 3875
Starting permutation 3876
Starting permutation 3877
Starting per

Starting permutation 4160
Starting permutation 4161
Starting permutation 4162
Starting permutation 4163
Starting permutation 4164
Starting permutation 4165
Starting permutation 4166
Starting permutation 4167
Starting permutation 4168
Starting permutation 4169
Starting permutation 4170
Starting permutation 4171
Starting permutation 4172
Starting permutation 4173
Starting permutation 4174
Starting permutation 4175
Starting permutation 4176
Starting permutation 4177
Starting permutation 4178
Starting permutation 4179
Starting permutation 4180
Starting permutation 4181
Starting permutation 4182
Starting permutation 4183
Starting permutation 4184
Starting permutation 4185
Starting permutation 4186
Starting permutation 4187
Starting permutation 4188
Starting permutation 4189
Starting permutation 4190
Starting permutation 4191
Starting permutation 4192
Starting permutation 4193
Starting permutation 4194
Starting permutation 4195
Starting permutation 4196
Starting permutation 4197
Starting per

Starting permutation 4480
Starting permutation 4481
Starting permutation 4482
Starting permutation 4483
Starting permutation 4484
Starting permutation 4485
Starting permutation 4486
Starting permutation 4487
Starting permutation 4488
Starting permutation 4489
Starting permutation 4490
Starting permutation 4491
Starting permutation 4492
Starting permutation 4493
Starting permutation 4494
Starting permutation 4495
Starting permutation 4496
Starting permutation 4497
Starting permutation 4498
Starting permutation 4499
Starting permutation 4500
Starting permutation 4501
Starting permutation 4502
Starting permutation 4503
Starting permutation 4504
Starting permutation 4505
Starting permutation 4506
Starting permutation 4507
Starting permutation 4508
Starting permutation 4509
Starting permutation 4510
Starting permutation 4511
Starting permutation 4512
Starting permutation 4513
Starting permutation 4514
Starting permutation 4515
Starting permutation 4516
Starting permutation 4517
Starting per

Starting permutation 4797
Starting permutation 4798
Starting permutation 4799
Starting permutation 4800
Starting permutation 4801
Starting permutation 4802
Starting permutation 4803
Starting permutation 4804
Starting permutation 4805
Starting permutation 4806
Starting permutation 4807
Starting permutation 4808
Starting permutation 4809
Starting permutation 4810
Starting permutation 4811
Starting permutation 4812
Starting permutation 4813
Starting permutation 4814
Starting permutation 4815
Starting permutation 4816
Starting permutation 4817
Starting permutation 4818
Starting permutation 4819
Starting permutation 4820
Starting permutation 4821
Starting permutation 4822
Starting permutation 4823
Starting permutation 4824
Starting permutation 4825
Starting permutation 4826
Starting permutation 4827
Starting permutation 4828
Starting permutation 4829
Starting permutation 4830
Starting permutation 4831
Starting permutation 4832
Starting permutation 4833
Starting permutation 4834
Starting per

0

In [4]:
import numpy as np
coord=np.load('/media/miplab-nas2/Data3/BMPD/hc_project/seed_to_voxels/spinalcord_icas_k9/C3/seed_coordinates.npy')

In [5]:
np.save('/media/miplab-nas2/Data3/BMPD/hc_project/seed_to_voxels/spinalcord_icas_k9/C3/seed_coordinates.npy',coord.T)

In [6]:
coord.T.shape

(4300, 3)